In [1]:
from bs4 import BeautifulSoup, Tag
import requests
import os.path
from tqdm.notebook import tqdm

In [2]:
def found_content_container(tag):
    if not tag.has_attr("data-test-id") :
        return False
    return tag["data-test-id"] == "content-container"

In [3]:
def get_start_index(content_container_child):
    start_index = 0
    found = False
    while not found:
        if isinstance(content_container_child[start_index], Tag):
            if content_container_child[start_index].attrs.get('id', "") == "question-answer-session":
                found = True
        start_index+=1
        if start_index == len(content_container_child) : 
            return -1
    start_index -=1
    return start_index

## Parameters

In [4]:
# change parameters here
dir_path_ticker = "data/sp600/ticker/"
dir_path_text = "data/sp600/text/"
list_tickers = os.listdir(dir_path_ticker)
#list_tickers.remove('.DS_Store') # for mac
list_tickers.remove('id_to_date')
print(list_tickers)

['AAON', 'AAN']


## Parsing

In [5]:
for ticker in tqdm(list_tickers):
    article_path = dir_path_ticker+ticker+"/"
    earning_call_path = dir_path_text+ticker
    earning_with_content_container_not_found = []
    earning_with_start_not_found = []
    for name in os.listdir(article_path):
        with open(article_path + name, "r") as file:
            html_content = file.read()
            soup  = BeautifulSoup(html_content)
            content_container = soup.find_all(found_content_container)
            if len(content_container) !=1:
                earning_with_content_container_not_found.append(name)
            else:
                content_container = content_container[0]
                content_container_child = [ child for child in content_container.children]
                start_index = get_start_index(content_container_child)
                if start_index == -1:
                    earning_with_start_not_found.append(name)
                else: 
                    earning_call_trasncript = ""
                    for tag in content_container_child[start_index:]:
                        earning_call_trasncript += tag.get_text()+'\n'
                    if not os.path.exists(earning_call_path):
                        os.makedirs(earning_call_path)
                        print("The new directory is created!")
                    if not os.path.isfile(earning_call_path+"/"+ name[:-5]):
                        with open(earning_call_path+'/'+ name[:-5],"x") as file:
                            file.write(earning_call_trasncript)

    print(f"We didn't find {len(earning_with_content_container_not_found)} content container out of {len(os.listdir(article_path))}")
    print(f"We didn't find {len(earning_with_start_not_found)} earning call start")

  0%|          | 0/2 [00:00<?, ?it/s]

We didn't find 1 content container out of 43
We didn't find 3 earning call start
We didn't find 1 content container out of 58
We didn't find 9 earning call start


Errors taken into account and that are, in our case, not errors are:
- 1 from the file last_time_scrap of the scrapping giving the last time in epoch we have scrapped
- Multiple file that are not written transcript but audio records
    
One can check which transcript id have error by printing earning_with_content_container_not_found or earning_with_start_not_found.